Double check if we want to continue with using player 2's probabilities

In [6]:
import numpy as np 
import itertools
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
import math
import kaleido


In [7]:
pip install -U kaleido   # This is to help with creating png, jpeg, whatever sort of image we want 

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 20.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
data = np.load('data/1001972621777634.npy', allow_pickle=True)
print(data)

[[0 0 0 1 1 0 1 1]
 [1 0 0 0 1 0 0 0]
 [1 1 0 1 1 1 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 1]
 [1 1 0 1 1 0 1 0]
 [0 1 0 1 0 0 0 1]
 [0 1 1 1 0 1 0 0]]


In [10]:
def create_heatmap(variation):
    '''
    This function takes in an 8x8 array and makes a heatmap using plotly.go. The heatmap displays the win ratios for the 
    2nd player, who will choose their pattern based on what the 1st player chose. The heatmap is scaled so that percentages 
    under 0.5 show up orange while those over 0.5 are green, with the midpoint 0.5 being yellow. The top left to bottom
    right diagonal should be None values which will show up grey, since these are matches that wouldn't occur (ex. RRR v RRR).
    From top down (and right to left), the order of the sequences is 'RRR', 'RRB', 'RBR', 'RBB', 'BRR', 'BRB', 'BBR', 'BBB'.

    Parameters: 
    variation: int, the variation of the game (1 for cards, 2 for tricks)
    '''
    
    data_folder = './data'  
    files = [file for file in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, file)) and file.endswith('.npy')]   # gets all the npy files right

    games_total = None

    for file in files:   # loading each of the game 
        file_path = os.path.join(data_folder, file)
        game_data = np.load(file_path, allow_pickle=True)  
        
        if games_total is None:
            games_total = game_data 
        else:
            games_total += game_data  

    total_iterations = len(files)   # THIS IS IMPORTANT / THIS GIVES TOTAL GAMES
    win_ratios = games_total / total_iterations  

    win_ratios_percentage = win_ratios * 100

    np.fill_diagonal(win_ratios, None)   # can change this back to original code from ariannas here

    variation_name = 'cards' if variation == 1 else 'tricks'
    
    fig = go.Figure(data=go.Heatmap(
        z=win_ratios,
        colorscale='Fall_r',
        hovertemplate="%{y}:%{x} win ratio <br />%{z}", 
        name="",
        text=win_ratios_percentage,
        texttemplate='%{text:.0f}',  
        x=['RRR', 'RRB', 'RBR', 'RBB', 'BRR', 'BRB', 'BBR', 'BBB'],
        y=['RRR', 'RRB', 'RBR', 'RBB', 'BRR', 'BRB', 'BBR', 'BBB'],
        hoverongaps=False
    ))

    fig.update_layout(
        title=f'Penney Game, variation {variation}: Player Two Win Ratio for {variation_name}<br> Total Non-Tie Games, Approx: {total_iterations}',
        title_x=0.5,
        title_y=0.9,
        title_font_size=20,
        xaxis=dict(title='Player Two Choice'),
        yaxis=dict(title='Player One Choice'),
        width=800,  # Wider for better viewing in HTML
        height=800
    )

    fig.update_traces(xgap=1, ygap=1)
    
    # Return the figure object for saving
    return fig  

In [12]:
def save_heatmap_as_html(fig, filename):
    fig.write_html(filename)
    print(f"Heatmap saved as HTML: {filename}")

In [16]:
def save_heatmap_as_png(fig, filename):
    fig.update_layout(
        width=600,  
        height=600,  
        margin=dict(l=40, r=40, t=120, b=40),  # this helps with separating title with matrix
        title_font_size=18,  
    )
    
    fig.write_image(filename, scale=2)  
    print(f"Heatmap saved as PNG: {filename}")


In [17]:
def master_heatmap_function(format_type):
    '''
    Master function to create heatmaps for both variations of the Penney Game and save them in the specified format.

    Parameters:
    format_type: str, format to save the heatmap ('html', 'png', or 'both')
    '''
    # Variation 1
    fig1 = create_heatmap(variation=1)
    
    # Variation 2
    fig2 = create_heatmap(variation=2)

    if format_type == 'html':
        save_heatmap_as_html(fig1, 'heatmap_variation_1.html')
        save_heatmap_as_html(fig2, 'heatmap_variation_2.html')
    elif format_type == 'png':
        save_heatmap_as_png(fig1, 'heatmap_variation_1.png')
        save_heatmap_as_png(fig2, 'heatmap_variation_2.png')
    elif format_type == 'both':
        save_heatmap_as_html(fig1, 'heatmap_variation_1.html')
        save_heatmap_as_png(fig1, 'heatmap_variation_1.png')
        save_heatmap_as_html(fig2, 'heatmap_variation_2.html')
        save_heatmap_as_png(fig2, 'heatmap_variation_2.png')
    else:
        print(f"Unsupported format type: {format_type}")

In [19]:
master_heatmap_function(format_type='both') 

Heatmap saved as HTML: heatmap_variation_1.html
Heatmap saved as PNG: heatmap_variation_1.png
Heatmap saved as HTML: heatmap_variation_2.html
Heatmap saved as PNG: heatmap_variation_2.png


NEED TO ASK: but what is ron looking for in the different kinds of heatmaps? is it just changing like the lengths and widths or whatever specs?

This shows the same charts just cause the data is the same for right now